# | NLP | LLM | LangChain | Multi-Step Reasoning 1 |

## Natural Language Processing (NLP) and Large Language Models (LLM) with LangChain and Building Multi-stage Reasoning Systems

![Learning](https://t3.ftcdn.net/jpg/06/14/01/52/360_F_614015247_EWZHvC6AAOsaIOepakhyJvMqUu5tpLfY.jpg)


# <b>1 <span style='color:#78D118'>|</span> Overview</b>

In this notebook we're going to create AI systems:
- Named `CommentatorMoodModeratorAI` will be a prototype AI self-commenting-and-moderating tool that will create new reaction comments to a piece of text with one LLM and use another LLM to critique those comments and flag them if they are negative. To build this we will walk through the steps needed to construct prompts and chains, as well as multiple LLM Chains that take multiple inputs, both from the previous LLM and external. 

## Learning Objectives

By the end of this notebook, you will be able to:
1. Build prompt template and create new prompts with different inputs
2. Create basic LLM chains to connect prompts and LLMs.
3. Construct sequential chains of multiple `LLMChains` to perform multi-stage reasoning analysis. 
4. Use langchain agents to build semi-automated systems with an LLM-centric agent.


<img src="https://deepsense.ai/wp-content/uploads/2023/10/LangChain-announces-partnership-with-deepsense.jpeg" alt="Learning" width="50%">





### Setup


In [4]:
!pip install openai==1.6.1 httpcore==1.0.2 httpx==0.26.0 typing-extensions==4.9.0 pydantic==1.9.0 better-profanity langchain

  Obtaining dependency information for openai==1.6.1 from https://files.pythonhosted.org/packages/e7/44/5ece9adb8b5943273c845a1e3200168b396f556051b7d2745995abf41584/openai-1.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.0.2 from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpx==0.26.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions==4.9.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.7 MB/s 

In [5]:
cache_dir = "./cache"

In [6]:
import pandas as pd
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

# <b>2 <span style='color:#78D118'>|</span> AI self-commenting-and-moderating</b>

## `CommentatorMoodModeratorAI` - A self moderating system for social media

In this section we will build an AI system that consists of two LLMs. 

-` CommentatorMood` will be an LLM designed to read in a social media post and create a new comment. 
- However, `CommentatorMood` can be moody at times so there will always be a chance that it creates a negative-sentiment comment... we need to make sure we filter those out. 
- Luckily, that is the role of `Moderator`, the other LLM that will watch what `CommentatorMood` says and flag any negative comments to be removed. 



### Step 1 - Building the CommentatorMood Prompt

To build `CommentatorMood` we will need it to be able to read in the social media post and respond as a commenter. 

We will use engineered prompts to take as an input two things, the first is the social media post and the second is whether or not the comment will have a positive sentiment. 

We'll use a random number generator to create a chance of the flag to be positive or negative in `CommentatorMood` response.


Prompt template:

In [7]:
# Let's start with the prompt template

from langchain import PromptTemplate
import numpy as np

# Our template for CommentatorMood will instruct it on how it should respond, and what variables (using the {text} syntax) it should use.
commentator_mood_template = """
You are a social media post commenter, you will respond to the following post with a {sentiment} response with a limit of 50 words. 
Post:" {social_post}"
Comment: 
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
commentator_mood_prompt_template = PromptTemplate(
    input_variables=["sentiment", "social_post"],
    template=commentator_mood_template,
)

Add parms to promt Template:

In [8]:
# Okay now that's ready we need to make the randomized sentiment
random_sentiment = "mean"
# We'll also need our social media post:
social_post = "I'm learning about LangChain in this Kaggle, I'm having a lot of fun learning! #AI It's super cool !"

# Let's create the prompt and print it out, this will be given to the LLM.
commentator_mood_prompt = commentator_mood_prompt_template.format(
    sentiment=random_sentiment, 
    social_post=social_post
)
print(f"Commentator Mood prompt:{commentator_mood_prompt}")

Commentator Mood prompt:
You are a social media post commenter, you will respond to the following post with a mean response with a limit of 50 words. 
Post:" I'm learning about LangChain in this Kaggle, I'm having a lot of fun learning! #AI It's super cool !"
Comment: 



### Step 2 - Building the CommentatorMood LLM

Note: We provide an option for you to use either Hugging Face or OpenAI. If you continue with Hugging Face, the notebook execution will take a long time (up to 10 mins each cell). If you don't mind using OpenAI, following the next markdown cell for API key generation instructions. 

For OpenAI,  we will use their GPT-3 model: `text-babbage-001` as our LLM. 


#### OPTIONAL: Use OpenAI's language model

If you'd rather use OpenAI, you need to generate an OpenAI key. 

Steps:
1. You need to [create an account](https://platform.openai.com/signup) on OpenAI. 
2. Generate an OpenAI [API key here](https://platform.openai.com/account/api-keys). 

Note: OpenAI does not have a free option, but it gives you $5 as credit. Once you have exhausted your $5 credit, you will need to add your payment method. You will be [charged per token usage](https://openai.com/pricing). 

**IMPORTANT**: It's crucial that you keep your OpenAI API key to yourself. If others have access to your OpenAI key, they will be able to charge their usage to your account! 


In [9]:
import os
os.environ["OPENAI_API_KEY"] = "<FILL>"

In [10]:
# # To interact with LLMs in LangChain we need the following modules loaded
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import OpenAI

llm = OpenAI(model="text-babbage-001")
## We can also use a model from HuggingFaceHub if we wish to go open-source!

#model_id = "EleutherAI/gpt-neo-2.7B"
#tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
#model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=cache_dir)
#pipe = pipeline(
#    "text-generation", 
#    model=model, 
#    tokenizer=tokenizer, 
#    max_new_tokens=512, 
#    device_map='auto'
#)
#llm = HuggingFacePipeline(pipeline=pipe)
commentator_mood_llm = llm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Step 3 - Building our Prompt-LLM Chain


We can simplify our input by chaining the prompt template with our LLM so that we can pass the two variables directly to the chain.


In [13]:
from langchain.chains import LLMChain
from better_profanity import profanity


commentator_mood_chain = LLMChain(
    llm=commentator_mood_llm,
    prompt=commentator_mood_prompt_template,
    output_key="commentator_mood_said",
    verbose=False,
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

# To run our chain we use the .run() command and input our variables as a dict
commentator_mood_said = commentator_mood_chain.run(
    {
        "sentiment": random_sentiment, 
        "social_post": social_post
    }
)

# Before printing what Commentator Mood said, let's clean it up:
cleaned_commentator_mood_said = profanity.censor(commentator_mood_said)
print(f"Commentator Mood said:{cleaned_commentator_mood_said}")

Commentator Mood said:
You are a ****. Learn about LangChain before you start commenting on other people's Kaggles. AI is nothing but a **** gimmick, and you know it.


### Step 4 - Building the second chain for our Moderator

In [19]:
#####################################
# 1 # We will build the prompt template
# Our template for Moderator will take CommentatorMood's comment and do some sentiment analysis.
moderator_template = """
You are Moderator, the moderator of an online forum, you are strict and will not tolerate any negative comments. You will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word for word.
Original comment: {commentator_mood_said}
Edited comment:
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["commentator_mood_said"],
    template=moderator_template,
)

#####################################
# 2 # We connect an LLM for Moderator, (we could use a slightly more advanced model 'text-davinci-003 since we have some more logic in this prompt).

#moderator_llm = llm
# Uncomment the line below if you were to use OpenAI instead
moderator_llm = OpenAI(model="text-davinci-003")

#####################################
# 3 # We build the chain for Moderator
moderator_chain = LLMChain(
    llm=moderator_llm, 
    prompt=moderator_prompt_template, 
    verbose=False
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

#####################################
# 4 # Let's run the chain with what CommentatorMood last said
# To run our chain we use the .run() command and input our variables as a dict
moderator_says = moderator_chain.run({"commentator_mood_said": commentator_mood_said})
# Let's see what moderator said...
print(f"Moderator says: {moderator_says}")

Moderator says: ***** ***** ***** ***** ***** ***** Kaggles. AI is nothing but a ***** *****, and you know it.


### Step 5 - Building our first Sequential Chain

In [37]:
from langchain.chains import SequentialChain

# The SequentialChain class takes in the chains we are linking together, as well as the input variables that will be added to the chain. These input variables can be used at any point in the chain, not just the start.
commentator_mood_moderator_chain = SequentialChain(
    chains=[commentator_mood_chain, moderator_chain],
    input_variables=["sentiment", "social_post"],
    verbose=True,
)

# We can now run the chain with our randomized sentiment, and the social post!
chain_commentator_mood_moderator_says = commentator_mood_moderator_chain.run({"sentiment": random_sentiment, "social_post": social_post})
print(f"Commentator Mood Moderator said:{chain_commentator_mood_moderator_says}")



> Entering new SequentialChain chain...

> Finished chain.
Commentator Mood Moderator said:***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** ***** *****
